# Homework 2

In [1]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.mixture import GaussianMixture
from scipy.stats import norm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydotplus
import seaborn as sns

## Load Data
The feature space is already lagged. Therefore, you do not need to lag the variables yourself. The data track these companies over 3 years (2018-2020). We will train the data in 2018, validate in 2019, and forecast 2020. Further instructions are given in the questions.


In [2]:
dat = pd.read_csv('hw2_data.csv')

In [3]:
dat

,permno,fyear,label,ret,acc,agr,cfp,ep,ni,op,...,grltnoa,conv,operprof,capxint,chpm,alm,hire,herf,bm_ia,me_ia
0,15417,2018.0,0,0.102609,0.053313,0.252472,0.253973,0.148027,0.095118,0.323725,...,0.123393,0.0,0.467402,0.089973,-0.016902,0.469687,-0.050260,0.553758,1.622149,0.017834
1,89031,2018.0,1,0.067922,-0.001165,0.215518,0.005945,0.002326,0.012251,0.126233,...,0.024414,0.0,0.140084,0.033813,-0.049337,0.084011,0.215316,0.759126,0.318117,0.127617
2,11154,2018.0,0,-0.071429,0.000948,0.032913,0.184267,0.024266,0.008571,0.155644,...,0.114552,1.0,0.163587,0.091550,-0.066428,0.144939,-0.035948,0.482831,2.015340,0.072533
3,13556,2018.0,0,-0.326014,0.045752,-0.095131,-1.258587,-1.340418,1.527239,-1.582117,...,0.117086,0.0,-2.972048,0.020384,1.954118,-0.075192,-0.234043,0.114982,0.965078,0.001005
4,14296,2018.0,0,0.249443,0.016353,-0.020653,0.519260,0.449600,0.854538,-0.183793,...,0.329045,0.0,0.982905,0.014257,0.821456,0.269721,-0.048576,0.223411,1.151220,0.039338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,14532,2020.0,-1,-0.080498,-0.064369,-0.050658,0.064593,0.028561,0.000000,-0.213658,...,0.000831,0.0,-0.226537,0.000677,0.000912,0.683570,-0.010638,0.383695,1.508840,0.011199
2239,41355,2020.0,0,0.015946,0.043579,0.147297,0.082415,0.064142,-0.030160,0.383195,...,0.002844,0.0,0.402497,0.011861,0.031440,0.238843,-0.027287,0.964868,0.199428,2.990846
2240,13983,2020.0,0,0.112274,0.003504,-0.219388,-0.345708,-0.429181,0.004651,0.196534,...,-0.196021,0.0,0.116354,0.021174,-0.419594,0.178488,-0.050000,0.505660,1.127711,0.139232
2241,47619,2020.0,0,-0.057480,0.007940,-0.033335,-0.010621,-0.123947,0.000000,0.096157,...,-0.016991,0.0,0.093207,0.034593,-0.014232,0.726728,-0.134865,0.242645,3.577947,0.009458


In [4]:
dat.shape

(2243, 56)

## Generative Learning

#### 1. Keep only the labels between -1 and 3. 
Split the data into Train-Validation-Test:
- Training data should contain features in 2018, do not forget to remove ‘label’
- Training labels should only contain ‘label’ in 2018
- Validation data should contain features in 2019, do not forget to remove ‘label’ • Validation labels should only contain ‘label’ in 2019
- Test data should contain features in 2020, do not forget to remove ‘label’
- Test labels should only contain ‘label’ in 2020

In [9]:
# filter label between -1 and 3
dat_1 = dat.loc[(dat['label'] >= -1) & (dat['label'] < 3)]

# Training data
train_feature = dat_1.loc[dat_1['fyear'] == 2018]
train_label = train_feature['label']
train_feature = train_feature.drop(columns=['label'])

# validation data
valid_feature = dat_1.loc[dat_1['fyear'] == 2019]
valid_label = valid_feature['label']
valid_feature = valid_feature.drop(columns=['label'])

# test data
test_feature = dat_1.loc[dat_1['fyear'] == 2020]
test_label = test_feature['label']
test_feature = test_feature.drop(columns=['label'])

#### 2. Compute and report the prior probabilities $\pi_j$ for all labels in the Training set.

In [11]:
all_label = pd.DataFrame(train_label).groupby('label').size()
all_label = pd.DataFrame(all_label, columns=['count'], index=all_label.index)
all_label['prior'] = all_label['count'] / sum(all_label['count'])

pi = all_label['prior']
pi

label
-1    0.070159
 0    0.820393
 1    0.088868
 2    0.020580
Name: prior, dtype: float64

#### 3. Using the Training set, calculate the likelihood for feature ‘ret’ to be 0.1 conditional on each value of the label $P_j = P(ret = 0.1|y = j)$ (Use the Normal PDF found in the following link: [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html])). Report the likelihood for each value of the label. You need to code this by hand, 0 points will be given if you use the pre-coded scipy function.

In [25]:
train = dat_1.loc[dat_1['fyear'] == 2018]
ret_mean = train.groupby(['label']).mean()['ret']
ret_var = train.groupby(['label']).var()['ret']

P = pd.DataFrame()
P['mean'] = ret_mean
P['var'] = ret_var
P['0.1_likelihood'] = norm.pdf((0.1 - P['mean']) / np.sqrt(P['var']))
P

,mean,var,0.1_likelihood
label,,,
-1,0.014262,0.013064,0.301106
0,0.010048,0.010146,0.267751
1,0.027497,0.012392,0.322697
2,-0.010655,0.008859,0.199889


#### 4. Use Guassian naive bayes from the scikit-learn library (found here:[scikitlearnfunction](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)) to classify the test data. Report the accuracy. You need to use the train+validation set.

In [26]:
gnb = GaussianNB()
y_pred = gnb.fit(
    pd.concat([train_feature, valid_feature], ignore_index=True), 
    pd.concat([train_label, valid_label], ignore_index=True)
    ).predict(test_feature)

test_accuracy = accuracy_score(test_label, y_pred)

print(f'test accuracy is {round(test_accuracy, ndigits=4)}')

test accuracy is 0.2887


#### 5. Compute the confusion matrix (as shown in the lectures) and report the top 2 pairs with most (absolute number) incorrect classifications.

In [18]:
mat = confusion_matrix(test_label, y_pred, labels=[i for i in range(-1, 3)])
mat

array([[ 1,  0,  0,  7],
       [ 1, 22,  2, 52],
       [ 0,  0,  1,  7],
       [ 0,  0,  0,  4]])

In [20]:
mat = confusion_matrix(test_label, y_pred, labels=[i for i in range(-1, 3)])

def find_n_max(matrix, n):

    pairs = []

    for i in range(n):

        max_incorrect = 0
        max_row = 0
        max_col = 0

        for row in range(len(matrix)):
            
            for column in range(len(matrix[0])):
                
                val = matrix[row, column]
                
                if (val > max_incorrect) & (row != column):

                    max_incorrect = val
                    max_row = row
                    max_col = column
                
        matrix[max_row, max_col] = 0 # change the biggest value to 0 and loop again
        pairs.append([max_row, max_col]) # row and column
        
    return pairs

find_n_max(mat, 2)

[[1, 3], [0, 3]]

Based on the data given by `pairs`, we know that the top 2 pairs with most incorrect are [2, 0] and [2, -1].

#### 6. Implement Gaussian Mixture model on the data as shown in class. 
Tune the covariance type parameter on the validation data. 

Use the selected value to compute the test accuracy. As always, train the model on train+validation data to compute the test accuracy. Train the model twice, the first model should use the covariance type that yielded the highest accuracy in the validation stage. The second model should use the covariance type that yielded the second highest accuracy in the validation stage. 

Comment on the accuracy on the test set of the the models you ran. 

Hint: Use ‘n components=3, init params=”kmeans”, random state=34’.

In [27]:
for cov in ['full', 'tied', 'diag', 'spherical']:

    clf = GaussianMixture(n_components=4, covariance_type=cov, init_params='kmeans', random_state=34)
    clf.means_init = np.array([train_feature[train_label == i].mean(axis=0) for i in range(-1, 3)])
    clf.fit(train_feature, train_label)
    pred = clf.predict(valid_feature)

    print ('Validation accuracy for covariance type '+ cov + ' = ' + str(accuracy_score(valid_label, pred)))

Validation accuracy for covariance type full = 0.6842105263157895
Validation accuracy for covariance type tied = 0.09952153110047847
Validation accuracy for covariance type diag = 0.02966507177033493
Validation accuracy for covariance type spherical = 0.1799043062200957


Based on the above results, we can see that the first model is `full` and the second is `tied`.

In [29]:
X = pd.concat([train_feature, valid_feature], ignore_index=True)
y = pd.concat([train_label, valid_label], ignore_index=True)

# first model
clf1 = GaussianMixture(n_components=4, covariance_type='full', init_params='kmeans', random_state=34)
clf1.means_init = np.array([X[y == i].mean(axis=0) for i in range(-1, 3)])
clf1.fit(X, y)
pred1 = clf1.predict(test_feature)

print(f'Test accuracy for covariance type full is {accuracy_score(test_label, pred1)}')

# second model
clf2 = GaussianMixture(n_components=4, covariance_type='tied', init_params='kmeans', random_state=34)
clf2.means_init = np.array([X[y == i].mean(axis=0) for i in range(-1, 3)])
clf2.fit(X, y)
pred2 = clf2.predict(test_feature)

print(f'Test accuracy for covariance type tied is {accuracy_score(test_label, pred2)}')

Test accuracy for covariance type full is 0.1958762886597938
Test accuracy for covariance type tied is 0.08247422680412371


As we can see, the test accuracy decreases sharply after we rerun the models using train+vaild data. 

We should be awared that the dataset was separated into three sets with same length. And there can be some similar pattern within this year thus causing overfit. 

After we use two-year data as the training data, the model could perform very bad.

#### 7. Bonus Question: Apply Linear Discriminant Analysis model on the train+validation data and report the accuracy obtained on test data. Report the transformation matrix (w) along with the intercept.

In [32]:
clf = LinearDiscriminantAnalysis(solver='eigen')
clf.fit(X, y)
y_pred = clf.predict(test_feature)
print(f'Test accuracy for discriminant analysis is {accuracy_score(test_label, y_pred)}')

Test accuracy for discriminant analysis is 0.7731958762886598


In [35]:
clf.coef_

array([[ 2.26680976e-03,  8.39045315e+03, -7.07258418e+02,
         8.99626216e+00,  6.34848515e+02, -5.23096410e+03,
         4.63106899e+03,  2.60911958e+02, -8.26841552e+01,
        -5.85193945e+02,  2.09892141e+01, -1.27152966e+01,
        -3.46604694e+02, -1.22320953e-02,  7.57087533e+01,
         1.96856917e+01, -4.39056956e+01,  8.86628905e+02,
        -1.54733704e+02, -3.38125142e+01,  5.89892746e+03,
        -4.74527389e+01, -3.84712575e+01,  8.69048746e+02,
         6.35946546e+02,  6.27384783e-05,  2.60933259e+01,
         1.58321785e+02, -2.62289300e+03,  1.22177948e+03,
         2.67774506e+03,  1.54828547e+01, -2.93670725e+02,
         1.34123417e+01,  9.83080979e+01, -4.71810209e+01,
         8.04493418e+00, -1.54480307e+01, -1.51340882e+00,
        -5.76851123e+01,  9.60669302e+01,  4.19385694e-02,
        -1.76722372e-01,  2.24674843e-01, -1.78178848e+02,
         2.39731427e+02, -2.88929038e+01,  1.01686606e+02,
        -4.34962272e+03,  1.71502384e+02, -9.82155462e+0

In [34]:
clf.intercept_

array([-8468247.51331814, -8468133.5827645 , -8468180.1162174 ,
       -8468016.42805395])

In [65]:
# simple computation of mean for the features in each class
mean_vectors = []
for cl in range(-1, 3):
    
    mean_vectors.append(np.mean(X[y == cl], axis=0))

In [80]:
# within class scatter matrix S_W 

S_W = np.zeros((53, 53)) # within class and between class 
for cl, mv in zip(range(-1, 3), mean_vectors):

    class_sc_mat = np.zeros((53, 53))                  # scatter matrix for every class

    for row in np.array(X[y == cl]):

        row, mv = row.reshape(53, 1), np.array(mv).reshape(53, 1) # make column vectors
        class_sc_mat += (row - mv).dot((row - mv).T)

    S_W += class_sc_mat                             # sum class scatter matrices

print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[ 3.37090280e+01  2.26961249e+00  2.08486945e+00 ... -2.87292510e+00
  -3.06798859e+01 -1.92177013e+01]
 [ 2.26961249e+00  8.20671455e+02  7.08876880e+00 ... -1.85227247e+00
   3.62100656e+01 -6.39508675e+01]
 [ 2.08486945e+00  7.08876880e+00  2.56472760e+02 ... -4.29710349e+00
  -4.74012464e+01 -9.26123489e+01]
 ...
 [-2.87292510e+00 -1.85227247e+00 -4.29710349e+00 ...  1.62216372e+02
   2.37906197e+01 -1.36474141e+02]
 [-3.06798859e+01  3.62100656e+01 -4.74012464e+01 ...  2.37906197e+01
   1.46418988e+03 -1.07294057e+03]
 [-1.92177013e+01 -6.39508675e+01 -9.26123489e+01 ... -1.36474141e+02
  -1.07294057e+03  3.90703411e+04]]


In [97]:
# between-class scatter matrix S_B

overall_mean = np.mean(X, axis=0)

S_B = np.zeros((53, 53))
for i, mean_vec in enumerate(mean_vectors):  

    n = np.array(X[y == i - 1]).shape[0]
    mean_vec = np.array(mean_vec).reshape(53, 1) # make column vector
    overall_mean = np.array(overall_mean).reshape(53, 1) # make column vector
    S_B += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

between-class Scatter Matrix:
 [[ 1.60678891e-01  1.08073875e-01  1.04489939e-01 ... -9.76099627e-02
   1.47898062e+00  1.08505095e+00]
 [ 1.08073875e-01  1.18967756e-01  1.72230009e-01 ...  1.27061446e-01
  -1.03087175e+00  3.22165415e+00]
 [ 1.04489939e-01  1.72230009e-01  4.44357839e-01 ...  4.78574905e-01
  -4.21202614e+00  7.76959298e+00]
 ...
 [-9.76099627e-02  1.27061446e-01  4.78574905e-01 ...  9.52773411e-01
  -9.88454415e+00  1.09363717e+01]
 [ 1.47898062e+00 -1.03087175e+00 -4.21202614e+00 ... -9.88454415e+00
   1.05613427e+02 -1.06462894e+02]
 [ 1.08505095e+00  3.22165415e+00  7.76959298e+00 ...  1.09363717e+01
  -1.06462894e+02  1.57831711e+02]]


In [103]:
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(53, 1)  

for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(53, 1)
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

W = np.hstack((eig_pairs[0][1].reshape(53, 1), eig_pairs[1][1].reshape(53, 1)))

W

array([[ 2.30980460e-02+0.j, -1.87114021e-02+0.j],
       [-1.51556576e-03+0.j, -1.47933131e-03+0.j],
       [-2.96345734e-02+0.j, -2.29568808e-02+0.j],
       [-1.11506276e-01+0.j,  4.98448069e-02+0.j],
       [ 1.25225281e-01+0.j, -4.85266274e-02+0.j],
       [ 6.08224241e-02+0.j,  4.64768781e-02+0.j],
       [-1.97884662e-03+0.j,  2.81086455e-03+0.j],
       [ 6.82809213e-03+0.j, -4.93035745e-03+0.j],
       [-4.93522368e-02+0.j,  5.74399164e-02+0.j],
       [-9.91386841e-03+0.j,  3.31033711e-03+0.j],
       [-1.79712796e-02+0.j, -1.71002894e-02+0.j],
       [ 1.52886473e-06+0.j,  7.05213261e-07+0.j],
       [ 8.58350842e-03+0.j,  5.59601876e-03+0.j],
       [ 5.49628505e-03+0.j,  6.74053740e-04+0.j],
       [ 2.36901594e-04+0.j, -6.27171761e-04+0.j],
       [ 4.21602112e-04+0.j,  9.87152123e-03+0.j],
       [ 4.54243613e-03+0.j,  6.86732317e-03+0.j],
       [ 1.18260316e-02+0.j,  8.56657543e-03+0.j],
       [-9.74268625e-01+0.j, -9.80222225e-01+0.j],
       [-4.28345799e-03+0.j, -3